<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguaje Natural

## Taller #9: Modelado de temas
`Fecha de entrega: Mayo 6, 2021. (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

In [2]:
import re
import pandas as pd 
from pprint import pprint

from nltk.corpus import stopwords
stopwords = stopwords.words('spanish')

import pyLDAvis.gensim_models
from gensim.models import LdaModel
from gensim.corpora import Dictionary

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

D:\ProgramFiles\Anaconda3.2020.11\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [3]:
# Cargar datos
path = './sampleFiles/reviews_vidjew_es.csv'
data = pd.read_csv(path)
data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games


###  `[12 pts]` Punto 1: Hacer pre-procesamiento del texto

In [7]:
# Definción de función pre-procesado sin concatenación
def preProcesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = texto.split() # Tokenizar
    texto = [palabra for palabra in texto if palabra not in stopwords]
    return texto

data["pre-procesado"] = data["review_body"].apply(lambda texto: preProcesado(texto))
data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,pre-procesado
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry,"[buen, buena, calidad, buena, presentación]"
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games,"[producto, perfecto, salir, casa, nintendo, sw..."
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games,"[funciona, nintendo, switch, forma, emparejarl..."
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games,"[recomendado, utilizo, pc, dan, ningún, proble..."
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games,"[cable, funciona, bien, podria, ser, mas, larg..."


###  `[10 pts]` Punto 2: Modelo de LDA

In [10]:

# Crear una represantación de los documentos en forma de diccionario
dictionary = Dictionary(data['pre-procesado'].values)

# Filtrar palabras muy frecuentes o infrecuentes
dictionary.filter_extremes(no_below=5, no_above=0.5) 

# Corpus
corpus = [dictionary.doc2bow(text) for text in data['pre-procesado'].values]

# Entrenar el modelo
model = LdaModel(corpus = corpus, id2word = dictionary, num_topics = 7, passes=10)


model.print_topics(num_words=10)

[(0,
  '0.063*"bien" + 0.059*"calidad" + 0.039*"precio" + 0.024*"cadena" + 0.021*"bonitos" + 0.017*"bastante" + 0.016*"buena" + 0.016*"mala" + 0.016*"aunque" + 0.012*"demasiado"'),
 (1,
  '0.042*"juego" + 0.030*"buen" + 0.024*"calidad" + 0.023*"bonito" + 0.020*"regalo" + 0.019*"gustado" + 0.017*"buena" + 0.017*"plata" + 0.014*"pulsera" + 0.014*"entrega"'),
 (2,
  '0.034*"bonito" + 0.033*"regalo" + 0.026*"queda" + 0.024*"pone" + 0.015*"parte" + 0.015*"viene" + 0.014*"pulsera" + 0.014*"bonita" + 0.013*"collar" + 0.012*"igual"'),
 (3,
  '0.030*"si" + 0.030*"bien" + 0.023*"calidad" + 0.021*"mando" + 0.016*"original" + 0.016*"precio" + 0.012*"buen" + 0.012*"buena" + 0.012*"jugar" + 0.011*"producto"'),
 (4,
  '0.062*"perfecto" + 0.028*"llegó" + 0.024*"producto" + 0.022*"tiempo" + 0.020*"juego" + 0.017*"amazon" + 0.014*"llego" + 0.014*"consola" + 0.014*"si" + 0.013*"después"'),
 (5,
  '0.042*"día" + 0.031*"juego" + 0.030*"producto" + 0.029*"llegado" + 0.016*"caja" + 0.015*"pedido" + 0.012*"es

###  `[25 pts]` Punto 3: Visualización de LDA

In [11]:
lda_display = pyLDAvis.gensim_models.prepare(model, corpus, dictionary)
pyLDAvis.display(lda_display)

###  `[3 pts]` Preguntas

- ¿Qué conclusiones puede sacar de los resultados de los últimos tres talleres, usando diferentes modelos pero los mismos datos?

* Analizando los resultados para el conjunto de datos propuesto y el problema de clasificación planteado (con base en el texto de opinión determinar una de dos clases: joyería o video juegos) se concluye que el tipo de modelos mas adecuado o que ofrece mejor desempeño es el supervisado y específicamente dentro de este tipo de modelos la máquina de vectores de soporte. Sin embargo el modelo LDA es significativamente útil en relación con el conjunto de datos para observar la relevancia de ciertos términos en las opiniones de los productos consignadas.